<a href="https://colab.research.google.com/github/GodoyLucas/king_of_rebirth/blob/main/Notebooks/King_of_Rebirth_Pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de desempenho dos players em função do input no campeonado de Call of Duty Warzone King of Rebirth

# Importanto bibliotecas e dados

In [9]:
import pandas as pd
import numpy as np

In [10]:
final = pd.read_csv('https://raw.githubusercontent.com/GodoyLucas/king_of_rebirth/main/Dados/open.csv', sep = ";")
final.head()

,player,input,partida,kills,colocacao
0,DETONAMACHO,controle,1,0,44
1,DETONAMACHO,controle,2,12,2
2,DETONAMACHO,controle,3,8,14
3,DETONAMACHO,controle,4,4,25
4,DETONAMACHO,controle,5,0,41


# Verificando condições gerais dos dados

In [11]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   player     220 non-null    object
 1   input      220 non-null    object
 2   partida    220 non-null    int64 
 3   kills      220 non-null    int64 
 4   colocacao  220 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 8.7+ KB


É possível verificar que, apesar de 4 registros nulos em 'kills' e 'colocacao', as 8 partidas dos 44 players estão presentes no conjunto de dados. 

Os dados nulos são então substituídos por "0" e o tipo dados dos campos são convertidos para inteiro.

In [12]:
final.fillna(value = {'kills': 0, 'colocacao': 0}, inplace = True)
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   player     220 non-null    object
 1   input      220 non-null    object
 2   partida    220 non-null    int64 
 3   kills      220 non-null    int64 
 4   colocacao  220 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 8.7+ KB


In [13]:
final = final.astype({'kills': int, 'colocacao': int})
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   player     220 non-null    object
 1   input      220 non-null    object
 2   partida    220 non-null    int64 
 3   kills      220 non-null    int64 
 4   colocacao  220 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 8.7+ KB


# Premissas de análise

- A final do campeonato foi realizada em 8 partidas;
- Das 8 partidas, apenas as 5 melhores foram usadas para a classificação geral do campeonato;
- O estudo contempla todas as partidas;
- O campeonato ocorreu no modo solo, ou seja, 1 pessoa por time, competição individual;
- 44 players participaram. Portanto, as colocações nas partidas vão de 1 a 44, não sendo possível empate;
- Nem todos os players participaram e/ou reportaram os resultados de todas as partidas. Houve caso do player não querer jogar, caso de ser desconectado da partida por motivo adverso e caso de não querer reportar o resultado de uma determinada partida. Os resultados faltantes decorrentes desses casos, resultados nulos ou resultados impossíveis dado o contexto serão estimados com base nos resultados do player nas demais partidas.
- Para que a lógica da planilha utilizada para consolidação dos resultados no dia do campeonato não quebrasse, resultados não reportados foram registrados com colocação 44. Este estudo procura identificar os registros legítimos para tratar os que assim só tenham sido reportados pelo uso da planilha.
  - Regristros com colocação 44 com zero kills só são considerados legítimos caso não haja outro registro nesta mesma colocação na partida;
  - Regristros com colocação 44 com mais de zero kills só são considerados legítimos caso não haja outro registro nesta mesma colocação com kills registradas na mesma partida;

Considerando a impossibilidade de empate em qualquer que seja a colocação, o ideal dos resultados seria não haver mais que um player por colocação em cada umas das partidas. Abaixo é possível observar tal ocorrencia, além da ocorrência da colocação "0":

In [14]:
registros_com_mais_de_um_player_por_colocacao = pd.DataFrame()

for partida in range(1, 9):
  for colocacao in range(0, 45):
    if final.query('partida == @partida  & colocacao == @colocacao').player.count() > 1:
      registros_com_mais_de_um_player_por_colocacao = pd.concat([registros_com_mais_de_um_player_por_colocacao, final.query('partida == @partida  & colocacao == @colocacao')])

registros_com_mais_de_um_player_por_colocacao.groupby(['partida', 'colocacao'])[['player']].count()

player
partida colocacao        
1       20              2
        27              2
        44              4
2       2               2
        44              5
3       44              4
4       40              2
        44              7
5       44              6

# Tratamento dos dados

Descrição do roteiro de tratamento dado as regras do jogo e as condições dos dados:

- Identificar e separar registros da colocação "0"
- Identificar e separar registros com mais de um player por colocação por partida;
- Identificar e separar registros legítimos da colocação "44";
- Consolidar registros legítimos sem  necessidade de tratamento;
- Consolidar registros para tratamento;
- Tratar registros;
- Consolidar dados tratados;

## Identificando e separando registros com colocação "0":

In [15]:
registros_com_colocacao_0 = final.query('colocacao == 0')
registros_com_colocacao_0

,player,input,partida,kills,colocacao


In [16]:
registros_sem_colocacao_0 = final[final.index.isin(list(set(final.index) - set(registros_com_colocacao_0.index)))]
registros_sem_colocacao_0.sort_values('colocacao')

,player,input,partida,kills,colocacao
166,FANTONFPS,mouse,2,6,1
218,ILUDIDO_FPS,controle,4,6,1
185,K4RAMBITT,controle,1,7,1
49,MAIKI,controle,5,10,1
122,VOORTZ,mouse,3,12,1
...,...,...,...,...,...
174,BEATRIZFS,controle,5,0,44
181,LERLINBR99,controle,2,0,44
191,WINX_FPS,controle,2,0,44
193,WINX_FPS,controle,4,0,44


## Identificando e separando registros com mais de um player por colocação por partida:

In [17]:
registros_com_mais_de_um_player_por_colocacao = pd.DataFrame()

for partida in range(1, 9):
  for colocacao in range(0, 45):
    if registros_sem_colocacao_0.query('partida == @partida  & colocacao == @colocacao').player.count() != 1:
      registros_com_mais_de_um_player_por_colocacao = pd.concat([registros_com_mais_de_um_player_por_colocacao, registros_sem_colocacao_0.query('partida == @partida  & colocacao == @colocacao')])

registros_com_mais_de_um_player_por_colocacao.sort_values('colocacao', ascending = False).head()

,player,input,partida,kills,colocacao
137,ZOIO GAMES13 (FOXTRO_RS),controle,3,0,44
92,LUUCASLUUCAS,controle,3,0,44
139,ZOIO GAMES13 (FOXTRO_RS),controle,5,0,44
114,SROGATA,controle,5,0,44
109,KROLPLAYS,controle,5,0,44


In [18]:
registros_sem_colocacao_0_e_apenas_um_player_por_colocacao = registros_sem_colocacao_0[registros_sem_colocacao_0.index.isin(list(set(final.index) - set(registros_com_mais_de_um_player_por_colocacao.index)))]
registros_sem_colocacao_0_e_apenas_um_player_por_colocacao.sort_values('colocacao')

,player,input,partida,kills,colocacao
122,VOORTZ,mouse,3,12,1
166,FANTONFPS,mouse,2,6,1
49,MAIKI,controle,5,10,1
185,K4RAMBITT,controle,1,7,1
218,ILUDIDO_FPS,controle,4,6,1
...,...,...,...,...,...
150,PATT_L,controle,1,0,42
26,FINEKSS,mouse,2,0,42
132,MILLERDETONA,mouse,3,0,42
203,PEDRINHOFPS,mouse,4,0,42


## Identificando e separando registros legítimos da colocação "44":

In [19]:
registros_legitimos_44_colocacao = pd.DataFrame()

for partida in range(1, 9):
  if registros_com_mais_de_um_player_por_colocacao.query('partida == @partida & colocacao == 44 & kills > 0').player.count() == 1:
    registros_legitimos_44_colocacao = pd.concat([registros_legitimos_44_colocacao, registros_com_mais_de_um_player_por_colocacao.query('partida == @partida & colocacao == 44 & kills > 0')])

registros_legitimos_44_colocacao

,player,input,partida,kills,colocacao
83,GAARA144HZ,mouse,4,1,44


## Consolidando registros legítimos sem  necessidade de tratamento;

In [20]:
registros_legitimos = pd.concat([registros_legitimos_44_colocacao, registros_sem_colocacao_0_e_apenas_um_player_por_colocacao])
registros_legitimos


,player,input,partida,kills,colocacao
83,GAARA144HZ,mouse,4,1,44
2,DETONAMACHO,controle,3,8,14
3,DETONAMACHO,controle,4,4,25
4,DETONAMACHO,controle,5,0,41
5,VIPSOAP,mouse,1,2,25
...,...,...,...,...,...
215,ILUDIDO_FPS,controle,1,1,41
216,ILUDIDO_FPS,controle,2,4,20
217,ILUDIDO_FPS,controle,3,2,16
218,ILUDIDO_FPS,controle,4,6,1


## Consolidando registros a serem tratados

In [21]:
registros_a_serem_tratados = final[final.index.isin(list(set(final.index) - set(registros_legitimos.index)))]
registros_a_serem_tratados.sort_values(['partida', 'colocacao'])

,player,input,partida,kills,colocacao
75,GREGGO,mouse,1,2,20
105,KROLPLAYS,controle,1,3,20
50,WILLOBRABO,controle,1,2,27
120,VOORTZ,mouse,1,4,27
0,DETONAMACHO,controle,1,0,44
20,FELPIIS32,mouse,1,0,44
80,GAARA144HZ,mouse,1,0,44
90,LUUCASLUUCAS,controle,1,0,44
1,DETONAMACHO,controle,2,12,2
21,FELPIIS32,mouse,2,4,2


## Tratando dados

In [22]:
base_para_tratamento = registros_legitimos[registros_legitimos.player.isin(list(registros_a_serem_tratados.player.unique()))].groupby(by = 'player')[['kills', 'colocacao']].mean().sort_values(by = 'kills', ascending = False)
base_para_tratamento

,kills,colocacao
player,,
CLARA FIERCE,7.500000,11.250000
FANTONFPS,5.750000,6.750000
SROGATA,5.000000,14.666667
VOORTZ,5.000000,17.000000
ZOIO GAMES13 (FOXTRO_RS),4.666667,27.000000
GREGGO,4.500000,27.250000
DETONAMACHO,4.000000,26.666667
WILLOBRABO,4.000000,27.000000
FABINHOFPS,3.750000,30.750000


In [23]:
registros_tratados = pd.merge(registros_a_serem_tratados, base_para_tratamento, how = 'inner', on = 'player')
registros_tratados.drop(['kills_x', 'colocacao_x'], axis = 1, inplace = True)
registros_tratados.rename(columns={'kills_y': 'kills', 'colocacao_y': 'colocacao'}, inplace = True)
registros_tratados

,player,input,partida,kills,colocacao
0,DETONAMACHO,controle,1,4.000000,26.666667
1,DETONAMACHO,controle,2,4.000000,26.666667
2,VIPSOAP,mouse,5,3.000000,17.250000
3,FABINHOFPS,controle,4,3.750000,30.750000
4,FELPIIS32,mouse,1,3.666667,20.333333
5,FELPIIS32,mouse,2,3.666667,20.333333
6,WILLOBRABO,controle,1,4.000000,27.000000
7,WILLOBRABO,controle,3,4.000000,27.000000
8,WILLOBRABO,controle,4,4.000000,27.000000
9,CLARA FIERCE,controle,4,7.500000,11.250000


## Consolidando registros para análise

In [24]:
final_tratado = pd.concat([registros_legitimos, registros_tratados])
final_tratado

,player,input,partida,kills,colocacao
83,GAARA144HZ,mouse,4,1.00,44.0
2,DETONAMACHO,controle,3,8.00,14.0
3,DETONAMACHO,controle,4,4.00,25.0
4,DETONAMACHO,controle,5,0.00,41.0
5,VIPSOAP,mouse,1,2.00,25.0
...,...,...,...,...,...
23,LERLINBR99,controle,2,2.75,24.5
24,WINX_FPS,controle,2,3.00,21.5
25,WINX_FPS,controle,3,3.00,21.5
26,WINX_FPS,controle,4,3.00,21.5


# Analise

In [25]:
final_tratado.groupby(by = 'input')[['kills', 'colocacao']].mean()

,kills,colocacao
input,,
controle,4.828986,20.805797
mouse,3.965833,20.381667


In [26]:
registros_legitimos.groupby(by = 'input')[['kills', 'colocacao']].mean()

,kills,colocacao
input,,
controle,5.010417,20.166667
mouse,3.956044,20.549451


In [27]:
final.groupby(by = 'input')[['kills', 'colocacao']].mean()

,kills,colocacao
input,,
controle,4.208333,24.208333
mouse,3.700000,21.790000
